## YOLO V3
- paper : An Incrementatl Improvement (2018)
- **Tech Report 느낌** 의 다른 Detection Algorithm 들의 방법들을 가져와 도움을 얻은 것들 및 다양한 경우 소개

<hr>

## 1. Bounding Box Prediction

![yolo_v3_boundingbox](img/yolo_v3_boundingbox.jpg)

### Bounding Box Prediction과 관련된 파라미터
- 쉽게 말해서 c_x, c_y가 존재(현재 위치느낌), yolo가 클러스터링을 통해서 anchor box를 생성함.
- 1) c_x, c_y : grid 셀의 좌상단의로부터의 좌표(offset)
    - YOLO는 이미지를 S*S grid 셀로 나눔
    - 예를 들어 7*7 grid 셀로 설정하면, 한 이미지에 7*7 = 49개의 셀이 생성

- 2) p_w, p_h
    - 앵커박스의 너비 및 높이
- 3) t_x, t_y, t_w, t_h
    - YOLO가 예측해야하는 물체의 좌표 값 (bounding box)
    - 시그모이드 함수를 통한 t_x, t_y를 0~1 사이의 값으로 초기화
- 4) b_x, b_y, b_w, b_h
    - 좌상단 offset으로 부터 얼마나 이동할것이고 (b_x, b_y)
    - 앵커박스의 너비와 높이를 얼만큼 비율로 조절할지 (b_w, b_h) 지수승을 통해 예측
    - 위 파라미터값을 조정한 최종 bounding box의 offsets 값들
    - 이를 토대로 GT(Ground Truth, 라벨 값)와 IOU를 계산

- 각각의 예측된 box마다 Objectness에 대한 점수를 logistic regression을 통해 구함
    - 즉 만약 예측된 어떤 box가 다른 box들보다 IOU가 높다면 해당값은 1
    - 만약 IOU가 제일 좋지는 않지만 특정한 Threshold보다 높다면 Faster R-CNN 방법을 사용해 무시
- YOLO v3는 IOU Threshold값을 0.5를 사용
- 하나의 box에 하나의 GT Box를 할당, box가 GT에 할당되지 않으면 좌표 및 클래스 예측에 손실을 주지 않고, Objectness에 대한 손실값만 적용

### Faster R-CNN Anchor Box vs Yolo v3 Anchor Box
- Faster R-CNN
    - 특정 Anchor가 Ground Truth와의 IOU가 0.7보다 크면 -> 해당 anchor box를 positive
    - 특정 Anchor가 Ground Truth와의 IOU가 0.3보다 작으면 -> 해당 anchor box는 non-positive
    - 나머지 구간 -> IOU가 가장 큰 anchor box를 positive로
    
- Yolo V3 
    - 특정 Anchor가 Ground Truth와의 IOU가 0.5보다 크면 -> 해당 Anchor Box를 Positive
    - 특정 Anchor가 Ground Truth와의 IOU가 0.5보다 작으면 -> 해당 경우 무시 및 Objectness에 대한 손실값 적용

  - b_x, b_y : 시그모이드 함수(sigmoid function)를 통해 t_x, t_y를 0~1 사이의 값으로 초기화
  
<hr>

## 2. Class Prediction
- 각각의 box는 클래스 예측시 Multi-label classification을 하게 됨
- 기존에는 class prediction시 softmax를 사용
    - but. softmax를 사용하는 것이 성능면에서 좋지 않다고 함.
    - yolo v3에서는 Softmax가 아닌 logistic classifiers가 좋다고 함
    - loss = binary cross-entropy 사용

- 문제점 및 접근 방법
    - 라벨들의 mutually inclusive 한 데이터셋 경우 더 적합
        - ex) Google의 Open Image Dataset 경우 label들(사람 & 여자)등이 겹치게 됨.
        - 이런 애매한 경우 model을 multilabel방식으로 접근하는 것이 더 좋은 접근방법
    - 라벨들이 mutually exclusive 한 경우 softmax가 좋음
    
<hr>

## 3. Predictions Across Scales
- 3종류의 다른 스케일의 box들을 예측 
- feature pyramid network의 컨셉과 유사한 방식으로 다양한 스케일에서 feature를 추출
- 출력값 : 3-d tensor => box정보, objectness, class 정보
    - N x N x [3*(4+1+80)] => (N, N, 255)
    - N x N의 Grid cell 갯수, anchor box의 갯수 3개, 4개의 box변수(box shift and scailing value), 1개의 objectness score, 80개의 classes
        
- (1) 이전 2번째 layer에서 feature map 가지고 와서 2배로 upscaling
- (2) 제일 앞단의 feature map을 가져와 둘을 concatenation하여 upsample(늘어난)된 feature map과 합침.
- (3) fine-grained information을 제일 앞단의 feature map으로 부터 얻음.
- (4) 최종적으로 3가지 multi-scale의 fine-grained feature를 이용한 box 예측

- yolo v3 에서도 기존 k-means를 이용한 anchor box dimension clustering 그대로 유지
    - yolo v3에서는 9개의 clsuter를 사용 및 3개의 scale에 대해 임의로 anchor box dimension 할당
    - COCO 기준
        - (10x13, 16x30, 33x23, 30x61, 62x45, 59x119, 116x90, 156x198, 373x326)
        
<hr>

## 4. Feature Extractor
- YOLO V3 에서는 Feature Extraction을 위한 새로운 network 모델(Darknet-53) 사용
- **YOLO V2, Darknet-19, Newfangled residual netwrok를 적절하게 섞어 사용.**
- 3x3과 1x1 conv 레이어를 성공적으로 적용, shortcut connection으로 인해 네트워크 자체가 많이 커짐
- 새로운 Layer는 Darknet-19모델보다 강력하며, resNet-101, 152 보다 조금더 효율적
- 

![yolov3_darknet-53](img/yolov3_darknet-53.jpg)

## Training
- full image 사용
- no hard negative mining
    - IOU 낮은값들: Best IOU 값 비율 조정 안해주고 그냥 학습
- multi-scale training
- data augmentation
- batch normalization

## 성능
- SSD와 성능은 비슷 하나 3배 더 빠름, RetinaNet보다는 성능이 조금 떨어짐

![yolov3_inference_time](img/yolov3_inference_time.png)


![yolo_v3_metrics](img/yolo_v3_metrics.jpg)

- 기존의 Detection Metric인 AP50을 사용시 RetinaNet과 비슷해지며, SSD 변형보다 강력해짐
- 과거 YOLO V1 이 작은 물체를 찾는데 좋았다면, YOLO V3은 작은 물체에는 조금약함
- 하지만 Accuracy와 speed를 같이 본다면 YOLO V3는 꽤나 이점이 있다.


## 추가실험내용
- 추가 실험을 진행 하였지만 좋은 결과를 내진 못했다.
- Anchor Box x,y offset predictions
    - linear activation을 사용한 다양한 box의 width, height의 x,y offset을 일반적인 anchor box예측 메커니즘 사용하려 했으나 오히려 모델의 안정성을 해쳤음
- Linear x,y predictions instead of logistic
    - logistic activation 대신 linear activation을 사용하여 다이렉트로 x,y의 offset값을 예측하려고 했지만 실패
    
- Focal loss
    - RetinaNet의 focal loss를 사용하였지만 잘 안되었다.

### YOLO Loss vs Retina loss
- RetinaNet Loss (Focal loss)
    - Detection 시스템에서 background라는 클래스를 따로 학습시킴
    - 이렇게 클래스를 background로 두게 되면, 한장의 이미지에서 Object가 있는 영역보다 없는 영역이 커짐
    - 이는 굉장히 큰 class unbalance유발
    - 그럴 경우 실제 Object가 있는 경우를 학습시키기 어려운 hard sample, Background 클래스의 경우는 학습시키기 쉬운 easy sample이다라고 생각
    - 기존에 Class Loss로 사용하던 Cross-Entropy를 hard sample과 easy sample에 따라 loss를 다르게 줌
    
- Yolo Loss
    - YOLO Loss는 Background, Object경우에 대한 Loss를 분할해서 주는 것이 아닌, Objectness score라는 부분을 따로 뽑고있음
    - 저자 왈) YOLO에 Focal Loss를 주는거 자체가 큰 의미는 없을거라고 생각

## 최종 결론
- YOLO V3는 빠르고 정확하다는 관점에서의 좋은 Detector이다.
- COCO Metric기준으로는 좋은편은 아니지만, 기존의 IOU 0.5기반 평가방법에서는 좋은 성능을 보임

## 참고문헌
- https://taeu.github.io/paper/deeplearning-paper-yolov3/
- https://dhhwang89.tistory.com/138